In [6]:
%config Completer.use_jedi = False
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
!pip install import_ipynb
import import_ipynb
from covid_analysis import *

In [14]:
world_geojson = json.load(open('countries.geojson', 'r'))
# covid_df = pd.read_csv('full_grouped.csv') already imported

covid_dt_df['confirmed_log'] = np.log10(covid_df['confirmed'] + 1e-10)
covid_dt_df

,country,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered,month,day,confirmed_log
0,Afghanistan,0,0,0,0,0,0,0,1,22,-10.000000
1,Albania,0,0,0,0,0,0,0,1,22,-10.000000
2,Algeria,0,0,0,0,0,0,0,1,22,-10.000000
3,Andorra,0,0,0,0,0,0,0,1,22,-10.000000
4,Angola,0,0,0,0,0,0,0,1,22,-10.000000
...,...,...,...,...,...,...,...,...,...,...,...
35151,West Bank and Gaza,10621,78,3752,6791,152,2,0,7,27,4.026165
35152,Western Sahara,10,1,8,1,0,0,0,7,27,1.000000
35153,Yemen,1691,483,833,375,10,4,36,7,27,3.228144
35154,Zambia,4552,140,2815,1597,71,1,465,7,27,3.658202


In [10]:
geojson_name_changes = {
    "United States of America": "US",
    "Democratic Republic of the Congo": "Congo (Kinshasa)" ,
    "Republic of Congo": "Congo (Brazzaville)",
    "United Republic of Tanzania":"Tanzania",
}
for feature in world_geojson['features']:
    old_name = feature['properties']['ADMIN']
    new_name = geojson_name_changes.get(old_name, old_name)
    feature['properties']['ADMIN'] = new_name
# csv_name_changes = {
#     "Burma" : "Myanmar",
#     "Cote d'Ivoire" : "Ivory Coast"
# }
# covid_df['country'] = covid_df['country'].replace(csv_name_changes)

In [11]:
#picking out the country names from the geojson
geojson_country_names = [feature['properties']['ADMIN'] for feature in world_geojson['features']]
# print(geojson_country_names)
# converting the csv and geojson country names in sets to compare
geojson_country_set = set(geojson_country_names)
csv_country_set = set(covid_df['country'])
#print(csv_country_set)
# making a set of common countries
common_countries = geojson_country_set.intersection(csv_country_set)

#filtering out the features list from the world_geojson 
filtered_geojson_features =  [feature for feature in world_geojson['features'] if feature['properties']['ADMIN'] in common_countries]

#making a new geojson to use
filtered_geojson = {'type' : 'FeatureCollection', 'features' : filtered_geojson_features}
# temp = set([feature['properties']['ADMIN'] for feature in filtered_geojson['features']])
# print(temp)

#making a new csv of countries
filtered_csv_countries = covid_df[covid_df['country'].isin(common_countries)]

#verifyling the equality of the 2 sets of data
# print( set(temp) - set(filtered_csv_countries['country'].unique()))

In [13]:
#assigning unique ids to each geojson country
country_id_map = {}
for feature in filtered_geojson['features']:
    feature['id'] = feature['properties']['ISO_A3']
    country_id_map[feature['properties']['ADMIN']] = feature['id']
#adding that id to csv, while supressing settingWithCopyWarning
with pd.option_context('mode.chained_assignment', None):
    filtered_csv_countries.loc[:, 'id'] = filtered_csv_countries['country'].apply(lambda x:country_id_map[x])

# country = 'Andorra'
# country_data = filtered_csv_countries[filtered_csv_countries['country'] == country]
# country_data
# date_to_check = '2020-07-23'
# date_data = filtered_csv_countries[filtered_csv_countries['date'] == date_to_check]
# filtered_geojson['features'][0].keys()

# fig = px.choropleth_mapbox(date_data,
#                     locations="id",
#                     geojson=filtered_geojson, 
#                     color='confirmed_log',
#                     hover_name='country',
#                     hover_data=['confirmed', 'deaths'],
#                     mapbox_style='carto-positron',
#                     zoom = 1.5,
#                     opacity = 0.7)
# fig.show()# Define slider steps
filtered_csv_countries.head(-10)

,date,country,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered,confirmed_log,id
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,-10.000000,AFG
1,2020-01-22,Albania,0,0,0,0,0,0,0,-10.000000,ALB
2,2020-01-22,Algeria,0,0,0,0,0,0,0,-10.000000,DZA
3,2020-01-22,Andorra,0,0,0,0,0,0,0,-10.000000,AND
4,2020-01-22,Angola,0,0,0,0,0,0,0,-10.000000,AGO
...,...,...,...,...,...,...,...,...,...,...,...
35140,2020-07-27,Tunisia,1455,50,1157,248,3,0,15,3.162863,TUN
35141,2020-07-27,Turkey,227019,5630,210469,10920,919,17,982,5.356062,TUR
35142,2020-07-27,US,4290259,148011,1325804,2816444,56336,1076,27941,6.632484,USA
35143,2020-07-27,Uganda,1128,2,986,140,13,0,4,3.052309,UGA
